In [1]:
%matplotlib inline


# nn.Transformer ve TorchText ile Dil Modelleme


[nn.Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html), [Attention](https://arxiv.org/pdf/1706.03762.pdf)_.
Diziden diziye birçok görev için kalite açısından transformatör modeli Tekrarlayan Sinir Ağları (RNN'ler) ile karşılaştırıldığında kalite açısından üstün olduğu kanıtlanmıştır. Daha fazla paralelleştirilebilr olmaları bunu mümkün kılar. `nn.Transformer` modülü tamamen dikkat mekanizması üzerine kuruludur.
[nn.MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html), 
[nn.TransformerEncoder](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html)_)


## Modeli tanımlayalım



 ``nn.TransformerEncoder`` modülünü dil modellemede kullanacağız. Dil modelleme görevi, belirli bir kelimenin (veya bir kelime dizisinin) olasılığı tahmin etmek için olasılıklar üretir.Bu işlem yapılırken kelime gömmeleri ve kelimelerin konumsal yerlerinin tespiti için konumsal bir katman eklenmesi gerekir.``nn.TransformerEncoder``
[nn.TransformerEncoderLayer için linke tıklayabilirsiniz.](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html). 



In [2]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

``PosionalEncoding`` modülü, 
dizideki belirteçlerin göreli veya mutlak konumu hakkında bazı bilgiler enjekte eder. Bu
konumsal kodlamalar, embeddings ile aynı boyuta sahiptir.




In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

## Load and batch data




Wikitext-2 datasetini kullanacağız. Buna erişebilmemiz için torchtext modülüne ihtiyacımız var. Aynı zamanda eğer torcdata kurulu değilse pip install ile indirmemiz gerekecek.

In [4]:
%%bash
pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 46.2 MB/s eta 0:00:00


Şimdi veri setimizdeki kelimeleri görseldeki mantığa göre hazırlamamız gerekir. Ve onları tensörlere çevirelim ki modelde kullanabilelim. Wikitext-2 içerisindeki bilinmeyen kelimer için de  `<unk>` kullanacağız.
Sıralı bir vektör verildiğinde ( a-z arası sutüna sahip, görseldeki 1B vektör ) `batchify()` işlevi verileri `batch_size` da verilen değer kadar sütunlara böler. Eğer verileri eşit olarak bölemez ise veriler sığacak şekilde kırpılır. 
Örneğin toplam uzunluk 26 batch_size = 4 ise alfabe 6 uzunluğunda 4 parçaya ayrılır.

\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

Bu gruplama işlemi daha fazla paralelleştirilebilir işlem yapmamıza olanak sağlar. Model her sutünu bağımsız olarak ele alır. Örneğin, bağımlılığı
yukarıdaki örnekte "G" ve "F" öğrenilemez.



In [5]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>']) 

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

### Girdi ve hedef dizisi oluşturma işlevleri




get_batch(), transformatör modeli için bir çift giriş-hedef dizisi üretir.Kaynak verileri bptt uzunluğunda parçalara ayırır.. Dil modelleme görevi için, modelin Hedef olarak aşağıdaki kelimelere ihtiyacı vardır.


In [6]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

## Örnek başlatma 




Model hiperparametreleri aşağıda tanımlanmıştır. kelime boyutu
sözcük nesnesinin uzunluğuna eşittir.




In [7]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

## Modelimizi çalıştıralım.



[CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html),
[SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html),
[StepLR](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.StepLR.html),
[nn.utils.clip_grad_norm](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html) bu belgeler modeli derlerken bize yardımcı olacaktır. Daha detaylı bilgi için linklere tıklayın.





In [8]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

Epoch'lardan sonra en iyi sonucu veren modeli kaydedelim ve lr ayarlamalarını planlanan şekilde yapalım.


In [9]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 29.36 | loss  8.09 | ppl  3274.96
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 15.15 | loss  6.87 | ppl   963.64
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 19.70 | loss  6.44 | ppl   623.29
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 16.69 | loss  6.30 | ppl   541.86
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 14.26 | loss  6.18 | ppl   484.98
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 14.25 | loss  6.15 | ppl   469.32
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 14.49 | loss  6.10 | ppl   447.18
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 14.74 | loss  6.10 | ppl   447.08
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 14.34 | loss  6.01 | ppl   408.69
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 14.40 | loss  6.01 | ppl   408.38
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 14.48 | loss  5.89 | ppl   362.58
| epoch   1 |  2400/ 

## Test veri kümesiyle en iyi modeli değerlendirin




In [10]:
test_loss = evaluate(model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.45 | test ppl   233.27
